# Advent of Code - Day 14

With apologies to anyone who's actually trying to read this code for some reason...

## Part 1

In [1]:
import re

In [2]:
def apply_mask(mask, int_in):
    # Convert to binary string, same length as mask
    bin_in=(bin(int_in)[2:]).zfill(len(mask))
    
    # Replace bits in binary with mask where necessary
    bin_out=''.join([m if m!='X' else bin_in[i]
                     for (i, m) in enumerate(mask)])
    
    return int(bin_out, base=2)

In [3]:
assert(apply_mask('XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X', 11)) == 73
assert(apply_mask('XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X', 101)) == 101
assert(apply_mask('XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X', 0)) == 64



Run the mask application against all the values in memory, then sum to give the output:

In [4]:
def day14_part1(FileIn):
    out_dict={}
    
    with open(FileIn) as fIn:
        for nl in fIn.readlines():
            m=re.match('mask = ([01X]+)', nl)
            if m:
                mask=m.group(1)
            else:
                m=re.match('mem\[(\d+)\] = (\d+)', nl)
                if m:
                    (reg, val)=(int(m.group(1)), int(m.group(2)))
                    out_dict[reg]=apply_mask(mask, val)
    return sum(out_dict.values())

In [5]:
assert day14_part1('data/day14_test')==165

In [6]:
day14_part1('data/day14_input')

6386593869035

## Part 2

I suspect this is the sort of naive approach that doesn't work in practice, but let's try anyway. Quickly eyeballing the input data suggests that there aren't generally too many `X`s per mask. (Can recheck if it doesn't work...)

Can hack this... convert the input into a list of masks, and then reuse `apply_mask`:

In [7]:
def expand_mask(mask_in):
    
    mask_in_a=mask_in.replace('0', '.')

    def replace_xs(input_str, replacement_str):
        rep_ls=list(replacement_str)
        return ''.join([c if c!='X' else rep_ls.pop(0)
                        for c in input_str])
    
    replacement_combos=[(bin(i)[2:]).zfill(mask_in.count('X')) 
                       for i in range(2**(mask_in.count('X')))]
    
    return [replace_xs(mask_in_a, replace_str).replace('.', 'X')
           for replace_str in replacement_combos]
    

In [8]:
expand_mask('0000010X1X0X1')

['XXXXX1X010X01',
 'XXXXX1X010X11',
 'XXXXX1X011X01',
 'XXXXX1X011X11',
 'XXXXX1X110X01',
 'XXXXX1X110X11',
 'XXXXX1X111X01',
 'XXXXX1X111X11']

Now we can apply these to the input values:

In [9]:
def day14_part2(FileIn):
    out_dict={}
    
    with open(FileIn) as fIn:
        for nl in fIn.readlines():
            m=re.match('mask = ([01X]+)', nl)
            if m:
                masks=expand_mask(m.group(1))
            else:
                m=re.match('mem\[(\d+)\] = (\d+)', nl)
                if m:
                    (reg, val)=(int(m.group(1)), int(m.group(2)))
                    # print((reg, val))
                    # reg_b=bin(int(reg))[2:].zfill(len(masks[0]))
                    for mask in masks:
                        reg_b=apply_mask(mask, int(reg))
                        out_dict[reg_b]=int(val)


    return sum(out_dict.values())

In [10]:
assert day14_part2('data/day14_testb')==208

In [11]:
day14_part2('data/day14_input')

4288986482164

OK, that worked quicker than I'd expected.

Done!